In [39]:
import pandas as pd
import numpy as np
import random

from collections import defaultdict
from itertools import chain
from glob import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [50]:
def get_train_image_path(folder, path='/opt/ml/input/data/train/images/'):
    ''' 각 이미지의 full_path를 얻는 함수
    
        folder(str) : 폴더 이름
        path(str) : train_image 폴더들의 상위 폴더 path, Default 설정해놨음
    
    '''
    
    file_path_list = glob(path + folder + '/*')
    return file_path_list


def figure_out_mask_label(file_path):
    ''' 마스크 착용 여부를 얻어내는 함수
    
        file_path(str) : file의 전체 경로 ex) 
            ex) ./input/data/train/images/000001_female_Asian ~~ /normal.jpg
    
    '''
    
    file_name = file_path.split('/')[-1]
    if 'incorrect' in file_name: return 'incorrect'
    elif 'mask' in file_name: return 'wear'
    else: return 'not_wear'


def get_label(label_dict, mask, gender, age):
    ''' label을 얻을 수 있는 함수
        
        label_dict(dict) : label값들을 가진 dictionary
        mask(str) : 마스크 착용 여부
        gender(str) : 성별
        age(int) : 나이
    
    '''
    
    if age < 30: age = 'young'
    elif (age >= 30 and age < 58) : age = 'middle'
    else: age = 'old'
    
    key = '_'.join([mask, gender, age])
    return label_dict[key]

    
def get_folder_path(full_path_list):
    ''' stratification함수에서 폴더명을 뽑기 위해 필요한 함수
    
        full_path_list(list): 이미지 경로가 담긴 리스트
    
    '''
    
    folder_path_list = []
    for full_path in full_path_list:
        folder_path = full_path.split('/')[-2] # 폴더명만 추출
        folder_path_list.append(folder_path)
    
    return folder_path_list

    
def stratification(df, infrequent_classes, ratio = 0.2):
    '''
        df : label값을 구하고 파일 기준으로 분류된 df
        infrequent_classes : 숫자가 적은 class 번호 순으로 정렬된 list
        ratio : 얻고자 하는 validation ratio
    '''
    
    total_valid_count = int(len(df) * ratio / 7) # valid용 folder의 개수
    valid_folder_list = [] # 여기에 valid용 folder명을 그룹마다 담을겁니다.
    count_summation = 0    # count_summation

    for class_num in infrequent_classes:
        # 만약 class_num이 마지막 infrequent_classes의 원소라면
        # total_valid_count를 맞추기 위해 그동안 쌓은 count_summation의 차만큼 뽑습니다.
        # why? 반올림으로 인해 완전히 나눠 떨어지지 않을 수도 있기 때문에
        if class_num == infrequent_classes[-1]:
            group_count = total_valid_count - count_summation
        else:
            group_count = round(label_count[class_num] * ratio)

        random.seed(42) # 복원을 위해 seed 설정
        group_df = df[df['label'] == class_num] # 현재 class_num을 가진 rows 추출
        index = random.sample(list(group_df.index), group_count) # 현재 group에서 뽑아야 하는 개수만큼 sampling
        group_full_path = df.iloc[index]['full_path'].values # index들의 full_path를 얻은 후
        group_folder_path = get_folder_path(group_full_path) # folder명만 추출 (리스트)
        valid_folder_list.append(group_folder_path) # valid_folder_list에 담고 
        count_summation += group_count # group_count를 쌓아간다.
        
    return valid_folder_list

In [58]:
train_df = pd.read_csv('/opt/ml/input/data/train/train.csv')
submission_df = pd.read_csv('/opt/ml/input/data/eval/info.csv')

In [60]:
# 각 폴더 내에 있는 파일 경로 읽어오기
# path_list는 array 형식으로 해당 폴더의 파일 경로 7개가 들어있음
train_df['path_list'] = train_df['path'].progress_apply(get_train_image_path)

In [61]:
train_df.head()

,id,gender,race,age,path,path_list
0,000001,female,Asian,45,000001_female_Asian_45,[/opt/ml/input/data/train/images/000001_female...
1,000002,female,Asian,52,000002_female_Asian_52,[/opt/ml/input/data/train/images/000002_female...
2,000004,male,Asian,54,000004_male_Asian_54,[/opt/ml/input/data/train/images/000004_male_A...
3,000005,female,Asian,58,000005_female_Asian_58,[/opt/ml/input/data/train/images/000005_female...
4,000006,female,Asian,59,000006_female_Asian_59,[/opt/ml/input/data/train/images/000006_female...


In [62]:
# 리스트화된 컬럼을 gender, age, path에 맞게 펼쳐준 뒤, merge하여 새로운 df 생성
gender_df = pd.DataFrame({'gender':np.repeat(train_df['gender'].values, train_df['path_list'].str.len()),
                          'full_path':np.concatenate(train_df['path_list'].values)
                         })

age_df = pd.DataFrame({'age':np.repeat(train_df['age'].values, train_df['path_list'].str.len()),
                       'full_path':np.concatenate(train_df['path_list'].values)
                      })

# 기존 DF의 path column의 이름을 folder로 변환
path_df = pd.DataFrame({'folder':np.repeat(train_df['path'].values, train_df['path_list'].str.len()),
                        'full_path':np.concatenate(train_df['path_list'].values)
                       })

# merge
new_df = pd.merge(gender_df, age_df, how='inner', on='full_path')
new_df = pd.merge(new_df, path_df, how='inner', on='full_path')


In [63]:
new_df.head()

,gender,full_path,age,folder
0,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45
1,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45
2,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45
3,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45
4,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45


In [64]:
# label Dictaionary는 라벨링을 할 때 사용됩니다.
label_dict = defaultdict(list)

label = 0
for mask in ('wear', 'incorrect', 'not_wear'):
    for gender in ('male', 'female'):
        for age in ('young', 'middle', 'old'):
            key = '_'.join([mask, gender, age])
            label_dict[key] = label
            label += 1

In [66]:
# 각 row마다 mask 여부 확인 후 mask column 생성
# incorrect, wear, not_wear
new_df['mask'] = new_df['full_path'].progress_apply(figure_out_mask_label)

# label 생성
# mask, gender, age 조합을 가지고 각 row마다 label 생성
new_df['label'] = new_df[['mask', 'gender', 'age']].progress_apply(lambda x: get_label(label_dict, x[0], x[1], x[2]), axis=1)

In [67]:
new_df.head()

,gender,full_path,age,folder,mask,label
0,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45,wear,4
1,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45,wear,4
2,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45,wear,4
3,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45,not_wear,16
4,female,/opt/ml/input/data/train/images/000001_female_...,45,000001_female_Asian_45,incorrect,10


In [68]:
# label을 count를 중복제거하고 오름차순 정렬
# 중복제거되는 label은 마스크 오착용 or 마스크 미착용 label일 것임
label_count = new_df['label'].value_counts().sort_values().drop_duplicates()

# label_count의 최초 6개는 incorrect or not_wear label이며, 가장 개수가 적은 label순으로 구성
# index로 함수에서 읽기 때문에 인덱스 추출
infrequent_classes = label_count[:6].index
label_count[:6]

14    179
17    257
13    313
6     554
16    670
15    727
Name: label, dtype: int64

In [69]:
# Validation ratio -> default는 0.2입니다.
ratio = 0.2

# 설정한 ratio대로 train, valid split
valid_folder_list = stratification(new_df, infrequent_classes, ratio)

# 함수로 얻어낸 valid_folder_path는 2D-array형식이며
# infrequent_classes개수에 맞게 6개 그룹으로 되어있음
# ex) [[classes1_folders], [classes2_folders], ... [classes6_folders]]

# 그러므로 작업 편의를 위해 1D-array로 변환
valid_folder_list = list(chain(*valid_folder_list))

In [70]:
# valid_df 생성
# new_df의 folder명이 valid_folder_path에 해당하면 추출
valid_df = new_df[new_df['folder'].isin(valid_folder_list)]

# trainset 분리 위해 valid_df의 index 추출
valid_index = valid_df.index

# train_df 생성
# new_df의 인덱스 중 valid_df의 인덱스가 아니면 train_df
train_index = [idx for idx in new_df.index if idx not in valid_index]
train_df = new_df.iloc[train_index]

In [71]:
# train_df, valid_df 저장
print(len(train_df), len(valid_df))

# path는 본인이 원하는 위치에 지정해주세요
train_df.to_csv('./train_df.csv', index=False)
valid_df.to_csv('./valid_df.csv', index=False)

15120 3780


In [76]:
df_train = pd.read_csv('./train_df.csv', index_col=0)
df_val = pd.read_csv('./valid_df.csv', index_col=0)

In [77]:
print(df_train['label'].value_counts())
print(df_val['label'].value_counts())

3     2910
4     2680
0     2215
1     1250
5     1030
2      715
9      582
15     582
10     536
16     536
6      443
12     443
13     250
7      250
11     206
17     206
14     143
8      143
Name: label, dtype: int64
3     725
4     670
0     555
1     315
5     255
2     180
15    145
9     145
10    134
16    134
12    111
6     111
7      63
13     63
11     51
17     51
14     36
8      36
Name: label, dtype: int64
